In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import preprocessing

## Model

In [2]:
def join(series):
    #return ' '.join( str(list(series)).strip('[]').split(', ') )
    return ' '.join( map(str, series) )

In [3]:
%run 'metrics.ipynb'
def model_evaluation(actual, prediction):
    return mapk(actual['ad_id'], prediction['ad_id'], k=12)

In [4]:
pd.read_csv("../generated/final/events_clicks_CTR_addocs_dist_cats_train.csv", nrows=10).head()

,display_id,document_id,timestamp,traffic_source,platform,day,hour,geo,ad_id,clicked,adsPerDisplay,clicksPerShows,addoc_id,dist,dist_cats
0,8,1330329,638,1.0,2.0,1,7,2765,95724,0,4,0.034785,1136820,0.537786,1.254512
1,8,1330329,638,1.0,2.0,1,7,2765,175694,0,4,0.202327,1392479,0.430210,0.098995
2,8,1330329,638,1.0,2.0,1,7,2765,280430,1,4,0.256853,1601512,0.245120,1.304837
3,8,1330329,638,1.0,2.0,1,7,2765,329774,0,4,0.052426,1652347,0.259652,1.192209
4,2657,1271490,185255,1.0,2.0,1,7,2765,70081,0,4,0.150145,933594,0.124739,0.409647


In [5]:
usecols = ['display_id', 'ad_id', 'clicksPerShows', 'dist', 'dist_cats', 'clicked']
#usecols = ['display_id', 'ad_id', 'clicksPerShows', 'platform', 'traffic_source', 'dist', 'clicked']
features = ['clicksPerShows', 'dist', 'dist_cats']
#features = ['clicksPerShows', 'clicksPerShowsM']
#features = ['p1', 'p2', 'p3', 't1', 't2', 't3', 'adsPerDisplay']

In [6]:
pd.read_csv("../generated/final/events_clicks_CTR_addocs_dist_cats_train.csv", nrows=10, usecols=usecols).head()

,display_id,ad_id,clicked,clicksPerShows,dist,dist_cats
0,8,95724,0,0.034785,0.537786,1.254512
1,8,175694,0,0.202327,0.430210,0.098995
2,8,280430,1,0.256853,0.245120,1.304837
3,8,329774,0,0.052426,0.259652,1.192209
4,2657,70081,0,0.150145,0.124739,0.409647


## Preparation for Validation

In [7]:
train_nrows, test_nrows = None, None #650000, 210000 #  None, None #

In [8]:
train_index = pd.read_csv("../generated/final/train_index.csv", nrows=train_nrows)['index'].ravel()
test_index = pd.read_csv("../generated/final/test_index.csv", nrows=test_nrows)['index'].ravel()

In [9]:
nrows = max(train_index[-1], test_index[-1])+1
nrows

87141731

In [10]:
train = pd.read_csv("../generated/final/events_clicks_CTR_addocs_dist_cats_train.csv", usecols = usecols, nrows=nrows)
train.count()

display_id        87141731
ad_id             87141731
clicked           87141731
clicksPerShows    87141731
dist              77941525
dist_cats         83237252
dtype: int64

In [11]:
dist_mean = train['dist'].mean()
dist_cats_mean = train['dist_cats'].mean()
train = train.fillna({'dist':dist_mean, 'dist_cats':dist_cats_mean}, inplace=True)

In [12]:
test = train.ix[test_index].copy()
train = train.ix[train_index].copy()
train.count()

display_id        65355241
ad_id             65355241
clicked           65355241
clicksPerShows    65355241
dist              65355241
dist_cats         65355241
dtype: int64

In [13]:
test.count()

display_id        21786490
ad_id             21786490
clicked           21786490
clicksPerShows    21786490
dist              21786490
dist_cats         21786490
dtype: int64

In [14]:
trainM = pd.DataFrame({'clicksPerShows_min':train.groupby('display_id')['clicksPerShows'].min(),
                       'clicksPerShows_max':train.groupby('display_id')['clicksPerShows'].max(),
                       'dist_min':train.groupby('display_id')['dist'].min(),
                       'dist_max':train.groupby('display_id')['dist'].max(),
                       'dist_cats_min':train.groupby('display_id')['dist_cats'].min(),
                       'dist_cats_max':train.groupby('display_id')['dist_cats'].max()
                      }).reset_index()
trainM.count()

display_id            12654568
clicksPerShows_max    12654568
clicksPerShows_min    12654568
dist_cats_max         12654568
dist_cats_min         12654568
dist_max              12654568
dist_min              12654568
dtype: int64

In [15]:
trainM['clicksPerShows_l'] = trainM['clicksPerShows_max'] - trainM['clicksPerShows_min']
trainM['dist_l'] = trainM['dist_max'] - trainM['dist_min']
trainM['dist_cats_l'] = trainM['dist_cats_max'] - trainM['dist_cats_min']
trainM.drop(['clicksPerShows_max', 'dist_max', 'dist_cats_max'], axis=1, inplace=True)
#trainM.drop(['clicksPerShows_max', 'dist_min'], axis=1, inplace=True)
trainM.count()

display_id            12654568
clicksPerShows_min    12654568
dist_cats_min         12654568
dist_min              12654568
clicksPerShows_l      12654568
dist_l                12654568
dist_cats_l           12654568
dtype: int64

In [16]:
print sum(trainM['clicksPerShows_l'] == 0)
print sum(trainM['dist_l'] == 0)
print sum(trainM['dist_cats_l'] == 0)

137
1250872
951046


In [17]:
trainM.loc[trainM['clicksPerShows_l'] == 0, 'clicksPerShows_l'] = 1
trainM.loc[trainM['dist_l'] == 0, 'dist_l'] = 1
trainM.loc[trainM['dist_cats_l'] == 0, 'dist_cats_l'] = 1

In [18]:
train = train.merge(trainM, on='display_id')
train.count()

display_id            65355241
ad_id                 65355241
clicked               65355241
clicksPerShows        65355241
dist                  65355241
dist_cats             65355241
clicksPerShows_min    65355241
dist_cats_min         65355241
dist_min              65355241
clicksPerShows_l      65355241
dist_l                65355241
dist_cats_l           65355241
dtype: int64

In [19]:
del trainM

In [20]:
train['clicksPerShows'] = ( train['clicksPerShows'] - train['clicksPerShows_min'] ) / train['clicksPerShows_l']
train['dist'] = ( train['dist'] - train['dist_min'] ) / train['dist_l']
train['dist_cats'] = ( train['dist_cats'] - train['dist_cats_min'] ) / train['dist_cats_l']
#train['dist'] = ( train['dist_max'] - train['dist'] ) / train['dist_l']
train.drop(['clicksPerShows_min', 'clicksPerShows_l', 'dist_min', 'dist_l'], axis=1, inplace=True)
train.drop(['dist_cats_min', 'dist_cats_l'], axis=1, inplace=True)
#train.drop(['clicksPerShows_min', 'clicksPerShows_l', 'dist_max', 'dist_l'], axis=1, inplace=True)
train.count()

display_id        65355241
ad_id             65355241
clicked           65355241
clicksPerShows    65355241
dist              65355241
dist_cats         65355241
dtype: int64

In [21]:
testM = pd.DataFrame({'clicksPerShows_min':test.groupby('display_id')['clicksPerShows'].min(),
                      'clicksPerShows_max':test.groupby('display_id')['clicksPerShows'].max(),
                      'dist_min':test.groupby('display_id')['dist'].min(),
                      'dist_max':test.groupby('display_id')['dist'].max(),
                      'dist_cats_min':test.groupby('display_id')['dist_cats'].min(),
                      'dist_cats_max':test.groupby('display_id')['dist_cats'].max()
                      }).reset_index()
testM.count()

display_id            4220025
clicksPerShows_max    4220025
clicksPerShows_min    4220025
dist_cats_max         4220025
dist_cats_min         4220025
dist_max              4220025
dist_min              4220025
dtype: int64

In [22]:
testM['clicksPerShows_l'] = testM['clicksPerShows_max'] - testM['clicksPerShows_min']
testM['dist_l'] = testM['dist_max'] - testM['dist_min']
testM['dist_cats_l'] = testM['dist_cats_max'] - testM['dist_cats_min']
testM.drop(['clicksPerShows_max', 'dist_max', 'dist_cats_max'], axis=1, inplace=True)

testM.count()

display_id            4220025
clicksPerShows_min    4220025
dist_cats_min         4220025
dist_min              4220025
clicksPerShows_l      4220025
dist_l                4220025
dist_cats_l           4220025
dtype: int64

In [23]:
print sum(testM['clicksPerShows_l'] == 0)
print sum(testM['dist_l'] == 0)
print sum(testM['dist_cats_l'] == 0)

54
417246
317644


In [24]:
testM.loc[testM['clicksPerShows_l'] == 0, 'clicksPerShows_l'] = 1
testM.loc[testM['dist_l'] == 0, 'dist_l'] = 1
testM.loc[testM['dist_cats_l'] == 0, 'dist_cats_l'] = 1

In [25]:
test = test.merge(testM, on='display_id')
del testM
test.count()

display_id            21786490
ad_id                 21786490
clicked               21786490
clicksPerShows        21786490
dist                  21786490
dist_cats             21786490
clicksPerShows_min    21786490
dist_cats_min         21786490
dist_min              21786490
clicksPerShows_l      21786490
dist_l                21786490
dist_cats_l           21786490
dtype: int64

In [26]:
test['clicksPerShows'] = ( test['clicksPerShows'] - test['clicksPerShows_min'] ) / test['clicksPerShows_l']
test['dist'] = ( test['dist'] - test['dist_min'] ) / test['dist_l']
test['dist_cats'] = ( test['dist_cats'] - test['dist_cats_min'] ) / test['dist_cats_l']
test.drop(['clicksPerShows_min', 'clicksPerShows_l', 'dist_min', 'dist_l'], axis=1, inplace=True)
test.drop(['dist_cats_min', 'dist_cats_l'], axis=1, inplace=True)
#test.drop(['clicksPerShows_min', 'clicksPerShows_l', 'dist_max', 'dist_l'], axis=1, inplace=True)
test.count()

display_id        21786490
ad_id             21786490
clicked           21786490
clicksPerShows    21786490
dist              21786490
dist_cats         21786490
dtype: int64

In [27]:
scaler = preprocessing.StandardScaler()
excpt = [] #['clicksPerShows', 'distM'] #, 'p1', 'p2', 'p3', 't1', 't2', 't3', 'adsPerDisplay']
for f in features:
    if not f in excpt:
        print f
        scaler.fit(train[[f]])
        train[f] = scaler.transform(train[[f]]) 
        test[f] = scaler.transform(test[[f]])

clicksPerShows
dist
dist_cats


In [28]:
train[features].head()

,clicksPerShows,dist,dist_cats
0,-1.123144,1.512830,0.882414
1,0.852239,0.581429,-1.305112
2,1.495124,-1.021097,0.977686
3,-0.915151,-0.895273,0.764467
4,-0.050373,-1.021097,-1.305112


## Validation

In [29]:
clf = linear_model.SGDClassifier(random_state=0, loss='log', fit_intercept=False, n_iter=5)
clf.fit(train[features], train['clicked'].ravel())

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=False, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=0, shuffle=True, verbose=0,
       warm_start=False)

In [30]:
clf.classes_

array([0, 1], dtype=int64)

In [31]:
clicks_test_sample = test[['display_id', 'ad_id', 'clicked']]
clicks_test_sample.count()

display_id    21786490
ad_id         21786490
clicked       21786490
dtype: int64

In [32]:
validation = clicks_test_sample[clicks_test_sample['clicked']==1].astype({'ad_id':str}).sort_values('display_id')
validation.count()

display_id    4220025
ad_id         4220025
clicked       4220025
dtype: int64

In [33]:
test[features].head()

,clicksPerShows,dist,dist_cats
0,1.495124,-1.021097,-1.305112
1,-1.123144,1.173232,-1.305112
2,-1.005127,0.640697,-1.305112
3,0.731841,0.071601,-1.305112
4,-0.956287,1.512830,-1.305112


In [34]:
test['res'] = pd.Series(clf.predict_proba(test[features])[:,1])
test.head()

,display_id,ad_id,clicked,clicksPerShows,dist,dist_cats,res
0,19972,120162,1,1.495124,-1.021097,-1.305112,0.713517
1,19972,122279,0,-1.123144,1.173232,-1.305112,0.368546
2,19972,168342,0,-1.005127,0.640697,-1.305112,0.381625
3,19972,176585,0,0.731841,0.071601,-1.305112,0.622357
4,19972,250080,0,-0.956287,1.512830,-1.305112,0.392844


In [35]:
clf.coef_, clf.intercept_[0]

(array([[ 0.57273608,  0.02214326, -0.06039855]]), 0.0)

In [36]:
prediction = test.sort_values(by=['display_id', 'res'], ascending=[True, False])
prediction.count()

display_id        21786490
ad_id             21786490
clicked           21786490
clicksPerShows    21786490
dist              21786490
dist_cats         21786490
res               21786490
dtype: int64

In [37]:
prediction=pd.DataFrame({ 'ad_id': prediction.groupby('display_id')['ad_id'].apply(join) }).astype({'ad_id':str}).reset_index()
prediction.count()

display_id    4220025
ad_id         4220025
dtype: int64

In [38]:
prediction.head()

,display_id,ad_id
0,1,279295 144739 42337 139684 296965 156824
1,7,300808 215967 105766
2,9,151028 19959 140940 104208
3,14,98270 143467 224171 288396
4,26,152193 285992


In [39]:
validation.head()

,display_id,ad_id,clicked
18142714,1,144739,1
14412118,7,105766,1
11270842,9,140940,1
377269,14,224171,1
18233315,26,152193,1


In [40]:
print 'clicksPerShows :', model_evaluation(validation, prediction)  #0.662869594202      0.588499968978      

clicksPerShows : 0.663048972212


In [41]:
#['clicksPerShows']                             0.662869594202       0.588499968978 
#['clicksPerShows', 'dist']                     0.66282443126        0.588638094838     -> 0.662839173432 (niter=20)
                                                                    #0.588785527156
# ['clicksPerShows', 'dist', 'dist_cats']       0.663048972212                     0.589767768832    
# ['clicksPerShows', 'dist_cats']                                    0.589561035359
    
    
                                      #         0.6628185045   +platform
                                     #          0.662806944913 +intercept
#['clicksPerShows', 'dist', 'adsPerDisplay']                         0.588542728654                                     
#['clicksPerShows', 'p1', 'p2', 'p3', 'dist']                        0.588165686987
#['clicksPerShows', 'dist', 't1', 't2', 't3']                        0.588181057654
#['clicksPerShows', 'p1', 'p2', 'p3']                                0.588499968978
#['clicksPerShows', 't1', 't2', 't3']                                0.588499968978 
#['clicksPerShows', 'adsPerDisplay']                                 0.588499968978


## Preparation for Training

In [9]:
train = pd.read_csv("../generated/final/events_clicks_CTR_addocs_dist_cats_train.csv", usecols = usecols)
train.count()

display_id        87141731
ad_id             87141731
clicked           87141731
clicksPerShows    87141731
dist              77941525
dist_cats         83237252
dtype: int64

In [7]:
trainM = pd.read_csv("../temp/trainM_full.csv")
trainM.count()

display_id            16874593
clicksPerShows_min    16874593
dist_cats_min         16874593
dist_min              16874593
clicksPerShows_l      16874593
dist_l                16874593
dist_cats_l           16874593
dtype: int64

In [10]:
input_file = "../generated/final/events_clicks_CTR_addocs_dist_cats_train.csv"
output_file = "../temp/train_merge.csv"
chunksize=5 * 10**6
how='inner'

In [11]:
first_time = True
size = 0
i = 0
    
f = open(output_file, 'w')

for train in pd.read_csv(input_file, chunksize=chunksize):
    
    train = train.merge(trainM, on='display_id')
    
    train['clicksPerShows'] = ( train['clicksPerShows'] - train['clicksPerShows_min'] ) / train['clicksPerShows_l']
    train['dist'] = ( train['dist'] - train['dist_min'] ) / train['dist_l']
    train['dist_cats'] = ( train['dist_cats'] - train['dist_cats_min'] ) / train['dist_cats_l']
    train.drop(['clicksPerShows_min', 'clicksPerShows_l', 'dist_min', 'dist_l'], axis=1, inplace=True)
    train.drop(['dist_cats_min', 'dist_cats_l'], axis=1, inplace=True)

    if first_time:
        train.to_csv(f, index = False, header=True)
        f.close()
        f = open(output_file, 'a')
        first_time = False
    else:
        train.to_csv(f, index = False, header=False)
    
    size+=chunksize
    i+=1
    print i, size
            
print 'Done'
    
f.close()

1 5000000
2 10000000
3 15000000
4 20000000
5 25000000
6 30000000
7 35000000
8 40000000
9 45000000
10 50000000
11 55000000
12 60000000
13 65000000
14 70000000
15 75000000
16 80000000
17 85000000
18 90000000
Done


In [13]:
del trainM

In [12]:
train = pd.read_csv(output_file, usecols = usecols)
train.count()

display_id        87141731
ad_id             87141731
clicked           87141731
clicksPerShows    87141731
dist              77941525
dist_cats         83237252
dtype: int64

In [13]:
dist_mean = train['dist'].mean()
dist_cats_mean = train['dist_cats'].mean()
train = train.fillna({'dist':dist_mean, 'dist_cats':dist_cats_mean}, inplace=True)
train.count()

display_id        87141731
ad_id             87141731
clicked           87141731
clicksPerShows    87141731
dist              87141731
dist_cats         87141731
dtype: int64

In [14]:
scaler = {}
excpt = [] #['clicksPerShows', 'distM'] #, 'p1', 'p2', 'p3', 't1', 't2', 't3', 'adsPerDisplay']
for f in features:
    if not f in excpt:
        print f
        scaler[f] = preprocessing.StandardScaler()
        scaler[f].fit(train[[f]])
        train[f] = scaler[f].transform(train[[f]]) 
        #test[f] = scaler.transform(test[[f]])

clicksPerShows
dist
dist_cats


In [15]:
train[features].head()

,clicksPerShows,dist,dist_cats
0,-1.123149,1.494529,0.856800
1,0.852229,0.501834,-1.422561
2,1.495111,-1.206151,0.956072
3,-0.915157,-1.072047,0.733902
4,-0.050381,-1.206151,-1.422561


### Training

In [16]:
clf = linear_model.SGDClassifier(random_state=0, loss='log', fit_intercept=False, n_iter=5)
clf.fit(train[features], train['clicked'].ravel())

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=False, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=0, shuffle=True, verbose=0,
       warm_start=False)

In [17]:
clf.classes_

array([0, 1], dtype=int64)

In [18]:
clf.coef_, clf.intercept_[0]

(array([[ 0.58280875,  0.02032292, -0.06478142]]), 0.0)

In [19]:
del train

### Preparation for Prediction

In [33]:
test = pd.read_csv("../generated/final/events_clicks_CTR_addocs_dist_cats_test.csv", usecols = usecols[:-1])
test.count()

display_id        32225162
ad_id             32225162
clicksPerShows    32225162
dist              27426869
dist_cats         30827903
dtype: int64

In [34]:
test = test.fillna({'dist':dist_mean, 'dist_cats':dist_cats_mean}, inplace=True)
test.count()

display_id        32225162
ad_id             32225162
clicksPerShows    32225162
dist              32225162
dist_cats         32225162
dtype: int64

In [35]:
testM = pd.DataFrame({'clicksPerShows_min':test.groupby('display_id')['clicksPerShows'].min(),
                      'clicksPerShows_max':test.groupby('display_id')['clicksPerShows'].max(),
                      'dist_min':test.groupby('display_id')['dist'].min(),
                      'dist_max':test.groupby('display_id')['dist'].max(),
                      'dist_cats_min':test.groupby('display_id')['dist_cats'].min(),
                      'dist_cats_max':test.groupby('display_id')['dist_cats'].max()
                      }).reset_index()
testM.count()

display_id            6245533
clicksPerShows_max    6245533
clicksPerShows_min    6245533
dist_cats_max         6245533
dist_cats_min         6245533
dist_max              6245533
dist_min              6245533
dtype: int64

In [36]:
testM['clicksPerShows_l'] = testM['clicksPerShows_max'] - testM['clicksPerShows_min']
testM['dist_l'] = testM['dist_max'] - testM['dist_min']
testM['dist_cats_l'] = testM['dist_cats_max'] - testM['dist_cats_min']
testM.drop(['clicksPerShows_max', 'dist_max', 'dist_cats_max'], axis=1, inplace=True)

testM.count()

display_id            6245533
clicksPerShows_min    6245533
dist_cats_min         6245533
dist_min              6245533
clicksPerShows_l      6245533
dist_l                6245533
dist_cats_l           6245533
dtype: int64

In [37]:
print sum(testM['clicksPerShows_l'] == 0)
print sum(testM['dist_l'] == 0)
print sum(testM['dist_cats_l'] == 0)

5545
894325
476289


In [38]:
testM.loc[testM['clicksPerShows_l'] == 0, 'clicksPerShows_l'] = 1
testM.loc[testM['dist_l'] == 0, 'dist_l'] = 1
testM.loc[testM['dist_cats_l'] == 0, 'dist_cats_l'] = 1

In [39]:
test = test.merge(testM, on='display_id')
del testM
test.count()

display_id            32225162
ad_id                 32225162
clicksPerShows        32225162
dist                  32225162
dist_cats             32225162
clicksPerShows_min    32225162
dist_cats_min         32225162
dist_min              32225162
clicksPerShows_l      32225162
dist_l                32225162
dist_cats_l           32225162
dtype: int64

In [40]:
test['clicksPerShows'] = ( test['clicksPerShows'] - test['clicksPerShows_min'] ) / test['clicksPerShows_l']
test['dist'] = ( test['dist'] - test['dist_min'] ) / test['dist_l']
test['dist_cats'] = ( test['dist_cats'] - test['dist_cats_min'] ) / test['dist_cats_l']
test.drop(['clicksPerShows_min', 'clicksPerShows_l', 'dist_min', 'dist_l'], axis=1, inplace=True)
test.drop(['dist_cats_min', 'dist_cats_l'], axis=1, inplace=True)
#test.drop(['clicksPerShows_min', 'clicksPerShows_l', 'dist_max', 'dist_l'], axis=1, inplace=True)
test.count()

display_id        32225162
ad_id             32225162
clicksPerShows    32225162
dist              32225162
dist_cats         32225162
dtype: int64

In [41]:
excpt = [] #['clicksPerShows', 'distM'] #, 'p1', 'p2', 'p3', 't1', 't2', 't3', 'adsPerDisplay']
for f in features:
    if not f in excpt:
        print f
        test[f] = scaler[f].transform(test[[f]])  #Use scaler from train

clicksPerShows
dist
dist_cats


In [42]:
test[features].head()

,clicksPerShows,dist,dist_cats
0,-0.596455,-0.632237,-1.422561
1,-0.092357,-0.131321,-1.422561
2,-1.123149,-0.107346,-1.422561
3,1.495111,1.494529,-1.422561
4,-0.706363,-0.431002,-1.422561


### Prediction

In [43]:
test['res'] = pd.Series(clf.predict_proba(test[features])[:,1])
test.head()

,display_id,ad_id,clicksPerShows,dist,dist_cats,res
0,16874904,51910,-0.596455,-0.632237,-1.422561,0.433321
1,16874904,151689,-0.092357,-0.131321,-1.422561,0.508914
2,16874904,156459,-1.123149,-0.107346,-1.422561,0.362482
3,16874904,165380,1.495111,1.494529,-1.422561,0.729856
4,16874904,256392,-0.706363,-0.431002,-1.422561,0.418660


In [44]:
prediction = test.sort_values(by=['display_id', 'res'], ascending=[True, False])
prediction.count()

display_id        32225162
ad_id             32225162
clicksPerShows    32225162
dist              32225162
dist_cats         32225162
res               32225162
dtype: int64

In [45]:
prediction=pd.DataFrame({ 'ad_id': prediction.groupby('display_id')['ad_id'].apply(join) }).astype({'ad_id':str}).reset_index()
prediction.count()

display_id    6245533
ad_id         6245533
dtype: int64

In [46]:
prediction.head()

,display_id,ad_id
0,16874594,170392 172888 162754 150083 180797 66758
1,16874595,8846 143982 30609
2,16874596,289122 289915 132820 11430 173005 288385 57197...
3,16874597,285834 305790 143981 182039 308836 155945 1809...
4,16874598,145937 335632 67292 250082


### Submission

In [47]:
prediction.to_csv("../submissions/solution_8.csv", index=False)